In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-07-21 21:14:45--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-07-21 21:14:51--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-07-21 21:14:52--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [2]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [3]:
!wget https://dl.fbaipublicfiles.com/glue/data/QQP-clean.zip

--2021-07-21 21:18:03--  https://dl.fbaipublicfiles.com/glue/data/QQP-clean.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41696084 (40M) [application/zip]
Saving to: ‘QQP-clean.zip’

QQP-clean.zip       100%[===================>]  39.76M  5.18MB/s    in 8.8s    

2021-07-21 21:18:22 (4.53 MB/s) - ‘QQP-clean.zip’ saved [41696084/41696084]



In [4]:
!unzip QQP-clean.zip

Archive:  QQP-clean.zip
   creating: QQP/
  inflating: QQP/train.tsv           
  inflating: QQP/dev.tsv             
  inflating: QQP/test.tsv            


In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/atuthvatullin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [453]:
torch.tensor([[1,2],[2,3]])

tensor([[1, 2],
        [2, 3]])

In [465]:
torch.nn.functional.normalize(torch.FloatTensor([[1,2], [2,2]]), p=2.0, dim=1, eps=1e-12, out=None)

tensor([[0.4472, 0.8944],
        [0.7071, 0.7071]])

In [3]:
import string
from collections import Counter
from typing import Dict, List, Tuple, Union, Callable

import nltk
import numpy as np
import math
import pandas as pd
import torch
import torch.nn.functional as F


# Замените пути до директорий и файлов! Можете использовать для локальной отладки. 
# При проверке на сервере пути будут изменены
glue_qqp_dir = 'QQP/'
glove_path = 'glove.6B.50d.txt'


class GaussianKernel(torch.nn.Module):
    def __init__(self, mu: float = 1., sigma: float = 1.):
        super().__init__()
        self.mu = mu
        self.sigma = sigma

    def forward(self, x):
        norm_sq = (x - self.mu)**2
        return np.exp(-norm_sq/(2*self.sigma**2))
    

class KNRM(torch.nn.Module):
    def __init__(self, embedding_matrix: np.ndarray, freeze_embeddings: bool, kernel_num: int = 21,
                 sigma: float = 0.1, exact_sigma: float = 0.001,
                 out_layers: List[int] = [10, 5]):
        super().__init__()
        self.embeddings = torch.nn.Embedding.from_pretrained(
            torch.FloatTensor(embedding_matrix),
            freeze=freeze_embeddings,
            padding_idx=0
        )

        self.kernel_num = kernel_num
        self.sigma = sigma
        self.exact_sigma = exact_sigma
        self.out_layers = out_layers

        self.kernels = self._get_kernels_layers()

        self.mlp = self._get_mlp()

        self.out_activation = torch.nn.Sigmoid()

    def _get_kernels_layers(self) -> torch.nn.ModuleList:
        
        kernels = torch.nn.ModuleList()
        
        # mu
        l_mus = [1]
        if self.kernel_num > 1:
            bin_size = (1-(-1)) / (self.kernel_num - 1)  # score range from [-1, 1]
            l_mus.append(1 - bin_size / 2)  # mu: middle of the bin
            for i in range(1, self.kernel_num - 1):
                l_mus.append(l_mus[i] - bin_size)
            l_mus = sorted(l_mus) # сортируем по возрастанию
        
        # sigma
        l_sigmas = [self.exact_sigma]
        if self.kernel_num > 1:
            l_sigmas = [self.sigma]*(self.kernel_num-1) + l_sigmas
    
        for i in range(self.kernel_num):
            gk = GaussianKernel(mu=l_mus[i], sigma=l_sigmas[i])
            kernels.append(gk)
            
        return kernels

    def _get_mlp(self) -> torch.nn.Sequential:
        
        self.layers = torch.nn.ModuleList()
        
        if len(self.out_layers) == 0:
            self.layers.append(torch.nn.Linear(len(self.kernels), 1))
            
        elif len(self.out_layers) == 1:
            self.layers.append(torch.nn.ReLU())
            self.layers.append(torch.nn.Linear(len(self.kernels), self.out_layers[0]))
            self.layers.append(torch.nn.ReLU())
            self.layers.append(torch.nn.Linear(self.out_layers[0], 1))
            
        else: 
            self.layers.append(torch.nn.ReLU())
            self.layers.append(torch.nn.Linear(len(self.kernels), self.out_layers[0]))
                
            for i in range(len(self.out_layers)):
                
                if i!=len(self.out_layers)-1:
                    self.layers.append(torch.nn.ReLU())
                    self.layers.append(torch.nn.Linear(self.out_layers[i], self.out_layers[i+1]))

                else: # последний слой
                    self.layers.append(torch.nn.ReLU())
                    self.layers.append(torch.nn.Linear(self.out_layers[-1], 1))
            
#             K->ReLU->10->ReLU->5->ReLU->1

        return torch.nn.Sequential(*self.layers)


    def sim_matrix(self,a, b, eps=1e-8):
        
#         cos = torch.nn.CosineSimilarity(dim=-1, eps=eps)
#         a = F.normalize(a)
#         b = F.normalize(b)
#         sim_mt = cos(a.unsqueeze(1), b)
        
        a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
        a_norm = a / torch.max(a_n, eps * torch.ones_like(a_n))
        b_norm = b / torch.max(b_n, eps * torch.ones_like(b_n))
        sim_mt = torch.mm(a_norm, b_norm.transpose(0, 1))

        return sim_mt

            
    def _get_matching_matrix(self, query: torch.Tensor, doc: torch.Tensor) -> torch.FloatTensor:
        
        vA = self.embeddings(query)
        vB = self.embeddings(doc) 
        output = []
        for i in range(len(vA)):
            s = self.sim_matrix(vA[i], vB[i])
            output.append(s)
        matrix = torch.stack(output)
        
        return matrix        
                                        
    def _apply_kernels(self, matching_matrix: torch.FloatTensor) -> torch.FloatTensor:
        KM = []
        for kernel in self.kernels:
            # shape = [B]
            K = torch.log1p(kernel(matching_matrix).sum(dim=-1)).sum(dim=-1)
            KM.append(K)
        # shape = [B, K]
        kernels_out = torch.stack(KM, dim=1)
        return kernels_out

    def predict(self, inputs: Dict[str, torch.Tensor]) -> torch.FloatTensor:
        # shape = [Batch, Left, Embedding], [Batch, Right, Embedding]
        query, doc = inputs['query'], inputs['document']
        # shape = [Batch, Left, Right]
        matching_matrix = self._get_matching_matrix(query, doc)
        # shape = [Batch, Kernels]
        kernels_out = self._apply_kernels(matching_matrix)
        # shape = [Batch]
        out = self.mlp(kernels_out)
        return out

    def forward(self, input_1: Dict[str, torch.Tensor], input_2: Dict[str, torch.Tensor]) -> torch.FloatTensor:
        logits_1 = self.predict(input_1)
        logits_2 = self.predict(input_2)

        logits_diff = logits_1 - logits_2

        out = self.out_activation(logits_diff)
        return out
    
class RankingDataset(torch.utils.data.Dataset):
    def __init__(self, index_pairs_or_triplets: List[List[Union[str, float]]],
                 idx_to_text_mapping: Dict[str, str], vocab: Dict[str, int], oov_val: int,
                 preproc_func: Callable, max_len: int = 30):
        self.index_pairs_or_triplets = index_pairs_or_triplets
        self.idx_to_text_mapping = idx_to_text_mapping
        self.vocab = vocab
        self.oov_val = oov_val
        self.preproc_func = preproc_func
        self.max_len = max_len

    def __len__(self):
        return len(self.index_pairs_or_triplets)

    def _tokenized_text_to_index(self, tokenized_text: List[str]) -> List[int]:
        return [self.vocab.get(i, self.oov_val) for i in tokenized_text]

    def _convert_text_idx_to_token_idxs(self, idx: int) -> List[int]:
        text = self.idx_to_text_mapping[idx]
        tokenized_text = self.preproc_func(text)
        token_idx = self._tokenized_text_to_index(tokenized_text)
        return token_idx

    def __getitem__(self, idx: int):
        pass


class TrainTripletsDataset(RankingDataset):
    def __getitem__(self, idx):
        
        triplet = self.index_pairs_or_triplets[idx]
                
        left_elem = {'query': self._convert_text_idx_to_token_idxs(triplet[0]),
                    'document': self._convert_text_idx_to_token_idxs(triplet[1])}
        
        right_elem = {'query': self._convert_text_idx_to_token_idxs(triplet[0]),
                    'document': self._convert_text_idx_to_token_idxs(triplet[2])}
        
        label = triplet[-1]
        
        return left_elem, right_elem, label


class ValPairsDataset(RankingDataset):
    def __getitem__(self, idx):
        
        duplet = self.index_pairs_or_triplets[idx]
                
        left_elem = {'query': self._convert_text_idx_to_token_idxs(duplet[0]),
                    'document': self._convert_text_idx_to_token_idxs(duplet[1])}

        label = duplet[-1]
        
        return left_elem, label


def collate_fn(batch_objs: List[Union[Dict[str, torch.Tensor], torch.FloatTensor]]):
    max_len_q1 = -1
    max_len_d1 = -1
    max_len_q2 = -1
    max_len_d2 = -1

    is_triplets = False
    for elem in batch_objs:
        if len(elem) == 3:
            left_elem, right_elem, label = elem
            is_triplets = True
        else:
            left_elem, label = elem

        max_len_q1 = max(len(left_elem['query']), max_len_q1)
        max_len_d1 = max(len(left_elem['document']), max_len_d1)
        if len(elem) == 3:
            max_len_q2 = max(len(right_elem['query']), max_len_q2)
            max_len_d2 = max(len(right_elem['document']), max_len_d2)

    q1s = []
    d1s = []
    q2s = []
    d2s = []
    labels = []

    for elem in batch_objs:
        if is_triplets:
            left_elem, right_elem, label = elem
        else:
            left_elem, label = elem

        pad_len1 = max_len_q1 - len(left_elem['query'])
        pad_len2 = max_len_d1 - len(left_elem['document'])
        if is_triplets:
            pad_len3 = max_len_q2 - len(right_elem['query'])
            pad_len4 = max_len_d2 - len(right_elem['document'])

        q1s.append(left_elem['query'] + [0] * pad_len1)
        d1s.append(left_elem['document'] + [0] * pad_len2)
        if is_triplets:
            q2s.append(right_elem['query'] + [0] * pad_len3)
            d2s.append(right_elem['document'] + [0] * pad_len4)
        labels.append([label])
    q1s = torch.LongTensor(q1s)
    d1s = torch.LongTensor(d1s)
    if is_triplets:
        q2s = torch.LongTensor(q2s)
        d2s = torch.LongTensor(d2s)
    labels = torch.FloatTensor(labels)

    ret_left = {'query': q1s, 'document': d1s}
    if is_triplets:
        ret_right = {'query': q2s, 'document': d2s}
        return ret_left, ret_right, labels
    else:
        return ret_left, labels


class Solution:
    def __init__(self, glue_qqp_dir: str, glove_vectors_path: str,
                 min_token_occurancies: int = 1,
                 random_seed: int = 0,
                 emb_rand_uni_bound: float = 0.2,
                 freeze_knrm_embeddings: bool = True,
                 knrm_kernel_num: int = 21,
                 knrm_out_mlp: List[int] = [],
                 dataloader_bs: int = 1024,
                 train_lr: float = 0.01, # 0.001
                 change_train_loader_ep: int = 10
                 ):
        self.glue_qqp_dir = glue_qqp_dir
        self.glove_vectors_path = glove_vectors_path
        
        self.glue_train_df = self.get_glue_df('train')
        self.glue_dev_df = self.get_glue_df('dev')
        self.dev_pairs_for_ndcg = self.create_val_pairs(self.glue_dev_df)
        self.min_token_occurancies = min_token_occurancies
        self.all_tokens = self.get_all_tokens(
            [self.glue_train_df, self.glue_dev_df], self.min_token_occurancies)

        self.random_seed = random_seed
        self.emb_rand_uni_bound = emb_rand_uni_bound
        self.freeze_knrm_embeddings = freeze_knrm_embeddings
        self.knrm_kernel_num = knrm_kernel_num
        self.knrm_out_mlp = knrm_out_mlp
        self.dataloader_bs = dataloader_bs
        self.train_lr = train_lr
        self.change_train_loader_ep = change_train_loader_ep

        self.model, self.vocab, self.unk_words = self.build_knrm_model()
        
        self.idx_to_text_mapping_train = self.get_idx_to_text_mapping(
            self.glue_train_df)
        self.idx_to_text_mapping_dev = self.get_idx_to_text_mapping(
            self.glue_dev_df)
        
        self.val_dataset = ValPairsDataset(self.dev_pairs_for_ndcg, 
              self.idx_to_text_mapping_dev, 
              vocab=self.vocab, oov_val=self.vocab['OOV'], 
              preproc_func=self.simple_preproc)
        self.val_dataloader = torch.utils.data.DataLoader(
            self.val_dataset, batch_size=self.dataloader_bs, num_workers=0, 
            collate_fn=collate_fn, shuffle=False)
        
        
        self.train_triplets_for_ndcg = self.sample_data_for_train_iter(self.glue_train_df, seed=42)
        
        self.train_dataset = TrainTripletsDataset(self.train_triplets_for_ndcg, 
              self.idx_to_text_mapping_train, 
              vocab=self.vocab, oov_val=self.vocab['OOV'], 
              preproc_func=self.simple_preproc)
        self.train_dataloader = torch.utils.data.DataLoader(
            self.train_dataset, batch_size=self.dataloader_bs, num_workers=0, 
            collate_fn=collate_fn, shuffle=False)
        

    def get_glue_df(self, partition_type: str) -> pd.DataFrame:
        assert partition_type in ['dev', 'train']
        glue_df = pd.read_csv(
            self.glue_qqp_dir + f'/{partition_type}.tsv', sep='\t', error_bad_lines=False, dtype=object)
        glue_df = glue_df.dropna(axis=0, how='any').reset_index(drop=True)
        glue_df_fin = pd.DataFrame({
            'id_left': glue_df['qid1'],
            'id_right': glue_df['qid2'],
            'text_left': glue_df['question1'],
            'text_right': glue_df['question2'],
            'label': glue_df['is_duplicate'].astype(int)
        })
        return glue_df_fin

    def hadle_punctuation(self, inp_str: str) -> str:
        trans_table={key:' ' for key in string.punctuation}
        out_str = inp_str.translate(str.maketrans(trans_table))
        return out_str

    def simple_preproc(self, inp_str: str) -> List[str]:

        out_str = self.hadle_punctuation(inp_str)
        out_str = out_str.lower()
        out_list = nltk.word_tokenize(out_str)
        return out_list
    
    def _filter_rare_words(self, vocab: Dict[str, int], min_occurancies: int) -> Dict[str, int]:
        vocab = {k: v for k, v in vocab.items() if v >= min_occurancies}
        return vocab
    
    def get_all_tokens(self, list_of_df: List[pd.DataFrame], min_occurancies: int) -> List[str]:
        texts_list = []
        for df in list_of_df:
            texts_list.append(df['text_left'].tolist())
            texts_list.append(df['text_right'].tolist())
            
        texts_list = [i for sl in texts_list for i in sl]
        
        texts_list = list(set(texts_list))
        
        texts_list = [self.simple_preproc(t) for t in texts_list]
        
        texts_list = [i for sl in texts_list for i in sl]
                
        tokens_dict = dict(Counter(texts_list))
        tokens_dict = self._filter_rare_words(tokens_dict, min_occurancies)
        all_tokens = list(tokens_dict.keys())
        
        return all_tokens
        
    def _read_glove_embeddings(self, file_path: str) -> Dict[str, List[str]]:
        embed_index = {}
        f = open(file_path, encoding='utf-8')
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embed_index[word] = coefs
        f.close()
        return embed_index
    
    def create_glove_emb_from_file(self, file_path: str, inner_keys: List[str],
                                   random_seed: int, rand_uni_bound: float
                                   ) -> Tuple[np.ndarray, Dict[str, int], List[str]]:
        
        np.random.seed(random_seed)
        emb_rand = np.random.uniform(-rand_uni_bound, rand_uni_bound, 50)

        unk_words = ["PAD", "OOV"]
        emb_matrix = np.tile(emb_rand, (len(inner_keys)+2, 1))
        emb_matrix[0] = np.zeros(50)
        
        embed_index = self._read_glove_embeddings(file_path)
        
        for i, word in enumerate(inner_keys):
            word_emb = embed_index.get(word)
            if word_emb is not None:
                emb_matrix[i+2] = word_emb
            else:
                unk_words.append(word)
           
        vocab = dict(zip(inner_keys, range(2,len(inner_keys)+2)))
        
        vocab.update({
            "PAD": 0,
            "OOV": 1
        })   
        
        self.emb_matrix = emb_matrix
        self.vocab = vocab
        self.unk_words = unk_words
        
        return emb_matrix, vocab, unk_words

    def build_knrm_model(self) -> Tuple[torch.nn.Module, Dict[str, int], List[str]]:
        emb_matrix, vocab, unk_words = self.create_glove_emb_from_file(
            self.glove_vectors_path, self.all_tokens, self.random_seed, self.emb_rand_uni_bound)
        torch.manual_seed(self.random_seed)
        knrm = KNRM(emb_matrix, freeze_embeddings=self.freeze_knrm_embeddings,
                    out_layers=self.knrm_out_mlp, kernel_num=self.knrm_kernel_num)
        return knrm, vocab, unk_words


    def sample_data_for_train_iter(self, inp_df: pd.DataFrame, seed: int
                                   ) -> List[List[Union[str, float]]]:
        
        inp_df_select = inp_df[['id_left', 'id_right', 'label']]
        
        t = inp_df_select.merge(inp_df_select, how='left', on='id_left')
        tt = t[(t.label_x==1)&(t.label_y==0)]
        tt['ones'] = 1
        ttt = tt[['id_left', 'id_right_x', 'id_right_y', 'ones']].values

        inp_df_select_unr = inp_df_select.copy()
        inp_df_select_unr.loc[:,'id_right'] = inp_df_select_unr['id_right'].sample(frac=1, random_state=42).values

        unr = inp_df_select_unr.merge(inp_df_select, on=['id_left', 'id_right'], how='left')
        unr = unr[unr.label_y.isnull()]

        ll = inp_df_select.merge(unr, how='left', on='id_left')
        ll['ones'] = 1
        lll = ll[['id_left', 'id_right_x', 'id_right_y', 'ones']].values

        itog = np.concatenate((ttt,lll))

        np.random.seed(seed)
        # np.random.shuffle(itog)
        itog_ones_ind = np.random.choice(len(itog), size=4500, replace=False)
        itog_ones = itog[itog_ones_ind]
        itog_ones = itog_ones.tolist()

        itog_zeros = []
        itog_zeros_ind = np.random.choice(len(itog), size=3500, replace=False)
        for i in itog_zeros_ind:
            tl = itog[i]
            q1 = tl[0]
            q2 = tl[2]
            q3 = tl[1]
            itog_zeros.append([q1, q2, q3, 0])

        final = itog_ones + itog_zeros
        np.random.shuffle(final)

        return final

    def create_val_pairs(self, inp_df: pd.DataFrame, fill_top_to: int = 15,
                         min_group_size: int = 2, seed: int = 0) -> List[List[Union[str, float]]]:
        inp_df_select = inp_df[['id_left', 'id_right', 'label']]
        inf_df_group_sizes = inp_df_select.groupby('id_left').size()
        glue_dev_leftids_to_use = list(
            inf_df_group_sizes[inf_df_group_sizes >= min_group_size].index)
        groups = inp_df_select[inp_df_select.id_left.isin(
            glue_dev_leftids_to_use)].groupby('id_left')

        all_ids = set(inp_df['id_left']).union(set(inp_df['id_right']))

        out_pairs = []

        np.random.seed(seed)

        for id_left, group in groups:
            ones_ids = group[group.label > 0].id_right.values
            zeroes_ids = group[group.label == 0].id_right.values
            sum_len = len(ones_ids) + len(zeroes_ids)
            num_pad_items = max(0, fill_top_to - sum_len)
            if num_pad_items > 0:
                cur_chosen = set(ones_ids).union(
                    set(zeroes_ids)).union({id_left})
                pad_sample = np.random.choice(
                    list(all_ids - cur_chosen), num_pad_items, replace=False).tolist()
            else:
                pad_sample = []
            for i in ones_ids:
                out_pairs.append([id_left, i, 2])
            for i in zeroes_ids:
                out_pairs.append([id_left, i, 1])
            for i in pad_sample:
                out_pairs.append([id_left, i, 0])
        return out_pairs

    def get_idx_to_text_mapping(self, inp_df: pd.DataFrame) -> Dict[str, str]:
        left_dict = (
            inp_df
            [['id_left', 'text_left']]
            .drop_duplicates()
            .set_index('id_left')
            ['text_left']
            .to_dict()
        )
        right_dict = (
            inp_df
            [['id_right', 'text_right']]
            .drop_duplicates()
            .set_index('id_right')
            ['text_right']
            .to_dict()
        )
        left_dict.update(right_dict)
        return left_dict

    def ndcg_k(self, ys_true: np.array, ys_pred: np.array, ndcg_top_k: int = 10) -> float:
        
        ys_ideal_sort_index = np.argsort(ys_true)[::-1]
        ys_ideal_true_sort = ys_true[ys_ideal_sort_index]
    
        if ndcg_top_k:
            ys_ideal_true_sort = ys_ideal_true_sort[:ndcg_top_k]

        ideal_dcg = 0
        for ix, e in enumerate(ys_ideal_true_sort):
            ideal_dcg += self.compute_gain(float(e), gain_scheme="exp2")/math.log2(ix+2)
    
        dcg_ = self.dcg(ys_true, ys_pred, gain_scheme="exp2", top_k=ndcg_top_k)
                
        return dcg_/ideal_dcg
    
    def compute_gain(self, y_value: float, gain_scheme: str) -> float:
        if gain_scheme=="const":
            return y_value
        elif gain_scheme=="exp2":
            # return 2**y_value - 1
            return pow(2, y_value) - 1
        else:
            return None
        
    def dcg(self, ys_true: np.array, ys_pred: np.array, gain_scheme: str, top_k: int) -> float:
        
        sort_indices = np.argsort(ys_pred)[::-1]
        ys_true_sort = ys_true[sort_indices]
                
        if top_k:
            ys_true_sort = ys_true_sort[:top_k]
            
        dcg_ = 0
        for ix, e in enumerate(ys_true_sort):
            dcg_ += self.compute_gain(float(e), gain_scheme)/math.log2(ix+2)

        return dcg_
    
    def valid(self, model: torch.nn.Module, val_dataloader: torch.utils.data.DataLoader) -> float:
        labels_and_groups = val_dataloader.dataset.index_pairs_or_triplets
        labels_and_groups = pd.DataFrame(labels_and_groups, columns=['left_id', 'right_id', 'rel'])
        
        all_preds = []
        for batch in (val_dataloader):
            inp_1, y = batch
            preds = model.predict(inp_1)
            preds_np = preds.detach().numpy()
            all_preds.append(preds_np)
        all_preds = np.concatenate(all_preds, axis=0)
        labels_and_groups['preds'] = all_preds
        
        ndcgs = []
        for cur_id in labels_and_groups.left_id.unique():
            cur_df = labels_and_groups[labels_and_groups.left_id == cur_id]
            ndcg = self.ndcg_k(cur_df.rel.values.reshape(-1), cur_df.preds.values.reshape(-1))
            if np.isnan(ndcg):
                ndcgs.append(0)
            else:
                ndcgs.append(ndcg)
        return np.mean(ndcgs)

    def train(self, n_epochs: int):
        
        opt = torch.optim.SGD(self.model.parameters(), lr=self.train_lr)
        criterion = torch.nn.BCELoss()
        
        self.model.train()
        
        for e in range(n_epochs):
            
            loss = 0
            
#             if (e%self.change_train_loader_ep == 0) & (e > 0):
                
#                 self.train_triplets_for_ndcg = self.sample_data_for_train_iter(self.glue_train_df, seed=42+e)

#                 self.train_dataset = TrainTripletsDataset(self.train_triplets_for_ndcg, 
#                       self.idx_to_text_mapping_train, 
#                       vocab=self.vocab, oov_val=self.vocab['OOV'], 
#                       preproc_func=self.simple_preproc)
#                 self.train_dataloader = torch.utils.data.DataLoader(
#                     self.train_dataset, batch_size=self.dataloader_bs, num_workers=0, 
#                     collate_fn=collate_fn, shuffle=False)
            
            for train_batch in self.train_dataloader:
                opt.zero_grad()
                inp_1, inp_2, batch_ys = train_batch
                batch_pred = self.model(inp_1, inp_2)
                batch_loss = criterion(batch_pred, batch_ys)
                batch_loss.backward()
                opt.step()
                loss += batch_loss.item()
                
            val_ndcg = self.valid(self.model, self.val_dataloader)
            print(f"Epoch {e} | BCELoss {np.mean(loss)} | NDCG@k {val_ndcg}")
         

In [2]:
s.train_triplets_for_ndcg = s.sample_data_for_train_iter(s.glue_train_df, seed=42+e)

NameError: name 's' is not defined

In [599]:
s.train_dataset = TrainTripletsDataset(s.train_triplets_for_ndcg, 
  s.idx_to_text_mapping_train, 
  vocab=s.vocab, oov_val=s.vocab['OOV'], 
  preproc_func=s.simple_preproc)

In [600]:
s.train_dataloader = torch.utils.data.DataLoader(
s.train_dataset, batch_size=s.dataloader_bs, num_workers=0, 
collate_fn=collate_fn, shuffle=False)

In [601]:
for train_batch in s.train_dataloader:
    print(train_batch)

({'query': tensor([[  13,   22,   21,  ...,    0,    0,    0],
        [  80, 7450,   15,  ...,    0,    0,    0],
        [  26,   22,   23,  ...,    0,    0,    0],
        ...,
        [  26,    2,   27,  ...,    0,    0,    0],
        [  26,  168,   27,  ...,    0,    0,    0],
        [  72, 2877,  541,  ...,    0,    0,    0]]), 'document': tensor([[   13,    75,    21,  ...,     0,     0,     0],
        [   13,    80,  1382,  ...,     0,     0,     0],
        [   13,   132, 11724,  ...,     0,     0,     0],
        ...,
        [   26,     2,    27,  ...,     0,     0,     0],
        [   26,   158,   205,  ...,     0,     0,     0],
        [   26,   168,    27,  ...,     0,     0,     0]])}, {'query': tensor([[  13,   22,   21,  ...,    0,    0,    0],
        [  80, 7450,   15,  ...,    0,    0,    0],
        [  26,   22,   23,  ...,    0,    0,    0],
        ...,
        [  26,    2,   27,  ...,    0,    0,    0],
        [  26,  168,   27,  ...,    0,    0,    0],
   

({'query': tensor([[  26,  168,  509,  ...,    0,    0,    0],
        [  13,  132,   23,  ...,    0,    0,    0],
        [  13,  132,   23,  ...,    0,    0,    0],
        ...,
        [  13,  132,   21,  ...,    0,    0,    0],
        [  13,   80, 7703,  ...,    0,    0,    0],
        [  13,  132,  137,  ...,    0,    0,    0]]), 'document': tensor([[4071, 1618,   44,  ...,    0,    0,    0],
        [  13,  132,   23,  ...,    0,    0,    0],
        [  13,  132,   21,  ...,    0,    0,    0],
        ...,
        [  26, 3000, 5380,  ...,    0,    0,    0],
        [  13,   22,   23,  ...,    0,    0,    0],
        [  13,   22,   23,  ...,    0,    0,    0]])}, {'query': tensor([[  26,  168,  509,  ...,    0,    0,    0],
        [  13,  132,   23,  ...,    0,    0,    0],
        [  13,  132,   23,  ...,    0,    0,    0],
        ...,
        [  13,  132,   21,  ...,    0,    0,    0],
        [  13,   80, 7703,  ...,    0,    0,    0],
        [  13,  132,  137,  ...,    0, 

- попробовать обучиться на 1 батч и смотреть на метрику на трейне, если все хорошо, то вы должны переобучиться, если же нет, то значит где-то в логике работы сети ошибка (в целом этот трюк часто помогает разобраться в работе сети и вообще хороший совет от Андрея Карпаты)

- зачем вы оставляете (retain_graph=true)? Это кушает память GPU, а нужно чтобы иметь возможность делать несколько backward‘ов после одного forward’а

- если оно учится без обнуления градиентов, то значит оно учится когда они копятся, может быть вы LR поставили очень маленький?

- ну-  в смысле, это же шаг оптимизации. Я его увеличил в 10 раз (1e-3 -> 1e-2). Вполне разумно что он влияет очень сильно: если он маленький, сетка учится медленно, оптимизация сходится медленно и еще может “застакаться” в плохом (высоком) локальном минимуме. Я это заметил просто по темпу уменьшения loss-функции. В нейронках это же ну чуть ли не основной параметр для тьюнинга (не считая архитектуры сети) 



- Да, я понимаю, просто я думал (и так было всегда), что без обновления градиентов сетка в принципе обучаться не будет, а тут наоборот получилось)
- ну тут если делать маленькие шаги, то за 10 эпох, оно фактически сделает 1 шаг для случая с 10xLR. Поэтому то, что оно обучалось - ну не везение, конечно, но в общем случае так не будет, особенно когда эпох много больше. Бывает, что градиенты таким образом копят (gradient accumulation trick), когда памяти для большого батча не хвататет, а хотелось бы, но их все равно обнуляют раз в несколько forward-pass’ов.


batch_loss.backward(retain_graph=True)
зачем вы оставляете (retain_graph=true)? 
Это кушает память GPU, а нужно чтобы иметь возможность 
делать несколько backward‘ов после одного forward’а.

In [18]:
%%time
s = Solution(glue_qqp_dir=glue_qqp_dir,
             glove_vectors_path=glove_path)

/home/atuthvatullin/environments/albert/lib/python3.6/site-packages/ipykernel_launcher.py:448: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 1min 33s, sys: 301 ms, total: 1min 33s
Wall time: 1min 33s


In [19]:
s.train(19)

Epoch 0 | BCELoss 74.69919818639755 | NDCG@k 0.5969478484574697
Epoch 1 | BCELoss 67.82673746347427 | NDCG@k 0.6732350804012821
Epoch 2 | BCELoss 10.391951382160187 | NDCG@k 0.7708105699599211
Epoch 3 | BCELoss 14.009461492300034 | NDCG@k 0.835115058585927
Epoch 4 | BCELoss 15.78466972708702 | NDCG@k 0.8772447190421852
Epoch 5 | BCELoss 17.472972750663757 | NDCG@k 0.8858484989145097
Epoch 6 | BCELoss 15.360442414879799 | NDCG@k 0.9026256244789225
Epoch 7 | BCELoss 13.435205459594727 | NDCG@k 0.9069991742313404
Epoch 8 | BCELoss 13.780339479446411 | NDCG@k 0.9140419785934341
Epoch 9 | BCELoss 12.085077375173569 | NDCG@k 0.9155337981537678
Epoch 10 | BCELoss 15.31329520046711 | NDCG@k 0.9213798461911914
Epoch 11 | BCELoss 10.350726053118706 | NDCG@k 0.9164321154732871
Epoch 12 | BCELoss 41.90720036625862 | NDCG@k 0.9316748454446152
Epoch 13 | BCELoss 37.52182373404503 | NDCG@k 0.9090078951925689
Epoch 14 | BCELoss 1.6010073274374008 | NDCG@k 0.9268422850278376
Epoch 15 | BCELoss 9.631582

In [14]:
torch.save(s.model.embeddings.state_dict(), 'knrm_embed.bin')

In [24]:
len(s.model.embeddings.state_dict()['weight'])

87164

In [17]:
torch.save(s.model.mlp.state_dict(), 'mlp.bin')

In [22]:
s.model.mlp.state_dict()

OrderedDict([('0.weight',
              tensor([[-0.0017,  0.1168, -0.1810, -0.1642, -0.0898,  0.0357,  0.0042,  0.2626,
                        0.1491,  0.2527,  0.1289,  0.1276, -0.0758, -0.0461, -0.0433, -0.0088,
                        0.0063,  0.0772,  0.0238,  0.5173,  1.0399]])),
             ('0.bias', tensor([0.1812]))])

In [20]:
from collections import OrderedDict

In [23]:
torch.save(OrderedDict([('0.0.weight',
              torch.Tensor([[-0.0017,  0.1168, -0.1810, -0.1644, -0.0903,  0.0340, -0.0185,  0.1554,
                        0.0101,  0.1091, -0.0145, -0.0103, -0.1788, -0.0660, -0.0424, -0.0073,
                        0.0063,  0.0771,  0.0298,  0.5381,  1.0739]])),
             ('0.0.bias', torch.Tensor([0.1812]))]), 'mlp_1.bin')

In [24]:
torch.save(OrderedDict([('0.0.weight',
              torch.Tensor([[-0.0017,  0.1168, -0.1810, -0.1642, -0.0898,  0.0357,  0.0042,  0.2626,
                        0.1491,  0.2527,  0.1289,  0.1276, -0.0758, -0.0461, -0.0433, -0.0088,
                        0.0063,  0.0772,  0.0238,  0.5173,  1.0399]])),
             ('0.0.bias', torch.Tensor([0.1812]))]), 'mlp_2.bin')

In [21]:
s.model.mlp

Sequential(
  (0): Linear(in_features=21, out_features=1, bias=True)
)

In [11]:
torch.save(s.model.mlp, 'mlp_1.bin')

In [16]:
import json
with open('vocab.json', 'w') as f:
    json.dump(s.vocab, f)

In [25]:
len(s.vocab)

87164

In [18]:
# load

# archive

In [307]:
knrm = KNRM(s.emb_matrix, s.freeze_knrm_embeddings)

In [440]:
np.random.seed(2)
vA = torch.LongTensor([[np.random.randint(1000) for i in range(31)] for a in range(1024)])
vB = torch.LongTensor([[np.random.randint(1000) for i in range(62)] for a in range(1024)])

In [441]:
vA = knrm.embeddings(vA)
vB = knrm.embeddings(vB)

In [442]:
vA.shape,vB.shape

(torch.Size([1024, 31, 50]), torch.Size([1024, 62, 50]))

In [523]:
def sim_matrix(a, b, eps=1e-8):
    """
    added eps for numerical stability
    """
#     a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
#     a_norm = a / torch.max(a_n, eps * torch.ones_like(a_n))
#     b_norm = b / torch.max(b_n, eps * torch.ones_like(b_n))
#     sim_mt = torch.mm(a_norm, b_norm.transpose(0, 1))
    cos = torch.nn.CosineSimilarity(dim=-1, eps=eps)
#     a = F.normalize(a)
#     b = F.normalize(b)
    sim_mt = cos(a.unsqueeze(1), b)
#     sim_mt = F.cosine_similarity(a,b)

    return sim_mt

In [524]:
output = []
for i in range(len(vA)):
    s = sim_matrix(vA[i], vB[i])
    output.append(s)

matrix1 = torch.stack(output)

In [526]:
def sim_matrix(a, b, eps=1e-8):
    """
    added eps for numerical stability
    """
    a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
    a_norm = a / torch.max(a_n, eps * torch.ones_like(a_n))
    b_norm = b / torch.max(b_n, eps * torch.ones_like(b_n))
    sim_mt = torch.mm(a_norm, b_norm.transpose(0, 1))

    return sim_mt

In [527]:
output = []
for i in range(len(vA)):
    s = sim_matrix(vA[i], vB[i])
    output.append(s)

matrix = torch.stack(output)

In [531]:
torch.allclose(matrix, matrix)

True

In [502]:
sim_matrix(F.normalize(vA[i]),F.normalize(vA[i]))

tensor([[ 1.0000e+00,  2.4906e-01,  6.2370e-01,  1.1181e-01,  5.6894e-01,
         -7.3797e-02,  6.0076e-01, -2.9285e-02,  7.3454e-01,  3.3433e-01,
          3.8078e-01,  5.0355e-01,  2.5614e-01,  4.0707e-01,  4.0707e-01,
          5.0197e-01,  6.0729e-01,  1.3733e-01,  4.2082e-01, -1.5886e-01,
          5.0278e-01,  7.0347e-01,  3.1847e-01,  3.8264e-01,  5.0305e-01,
          5.8528e-01,  1.7646e-01, -1.8621e-01,  7.0743e-01,  8.5510e-01,
         -1.1885e-01],
        [ 2.4906e-01,  1.0000e+00,  2.9212e-02,  2.2471e-01,  2.9357e-01,
          6.6197e-03,  5.7279e-02,  1.1422e-01,  4.7118e-02, -5.6208e-02,
          1.1162e-01,  3.8129e-01, -6.8250e-02, -1.8178e-01, -1.8178e-01,
         -1.7340e-02,  1.7689e-01, -2.3004e-02, -2.0182e-02,  2.5926e-01,
         -8.6881e-02,  1.2807e-02,  2.6289e-01,  1.4439e-01,  8.7400e-02,
          1.9350e-01, -4.3373e-02,  1.9095e-01, -1.1387e-01,  2.6632e-01,
          2.1493e-01],
        [ 6.2370e-01,  2.9212e-02,  1.0000e+00,  2.1255e-01,  8.49

In [498]:
x = torch.randn((4, 3, 32))
x
x = F.normalize(x, dim=0, p=2)
x.shape

torch.Size([4, 3, 32])

In [467]:
matrix[0]

tensor([[ 0.3910,  0.6785,  0.6430,  ...,  0.6427,  0.4616,  0.7578],
        [ 0.4815,  0.5720,  0.4883,  ...,  0.3487,  0.6591,  0.5531],
        [-0.1119, -0.2948, -0.2491,  ..., -0.0994, -0.2847, -0.1486],
        ...,
        [-0.1119, -0.2948, -0.2491,  ..., -0.0994, -0.2847, -0.1486],
        [ 0.1997,  0.4433,  0.6723,  ...,  0.4092,  0.5331,  0.5286],
        [ 0.2680,  0.3634,  0.3680,  ...,  0.3556,  0.2484,  0.4183]])

In [481]:
torch.nn.functional.normalize(matrix[0][0], dim=0, p=2.0)

tensor([ 0.0946,  0.1642,  0.1556,  0.2153,  0.1043,  0.1197,  0.1314,  0.0906,
         0.1719,  0.1469,  0.1738,  0.1456,  0.0867,  0.0175,  0.1745,  0.1191,
         0.1545,  0.0821,  0.1410, -0.1081,  0.1075,  0.0407,  0.1045,  0.1702,
         0.1156,  0.1103,  0.1524,  0.1199,  0.1738,  0.1290,  0.0066,  0.0918,
         0.1095,  0.1372,  0.1372,  0.0785,  0.1056,  0.1402,  0.1935,  0.0942,
         0.0620,  0.0552,  0.1264,  0.0909,  0.0365, -0.0589,  0.1300,  0.1306,
         0.0785,  0.1878,  0.1375,  0.1336,  0.1744,  0.0718,  0.1149, -0.0243,
         0.0704,  0.1107,  0.1905,  0.1555,  0.1117,  0.1833])

In [374]:
cos = torch.nn.CosineSimilarity(dim=-1, eps=1e-8)
matrix = cos(vA[1].unsqueeze(1), vB[1])
matrix

tensor([[ 0.2184,  0.0017,  0.2983,  ...,  0.4810, -0.1870,  0.2915],
        [-0.0558, -0.1941,  0.1702,  ...,  0.1114, -0.1398,  0.0626],
        [ 0.0960,  0.1344, -0.1712,  ..., -0.0192, -0.1004, -0.1435],
        ...,
        [ 0.1518, -0.0945,  0.4693,  ...,  0.6139, -0.0319,  0.1021],
        [ 0.2096, -0.1173,  0.4186,  ...,  0.4406,  0.0737,  0.1675],
        [-0.2742, -0.1711, -0.0318,  ..., -0.2129,  1.0000, -0.1324]])

In [377]:
cos(vA[1][2], vB[1][2])

tensor(-0.1712)

In [361]:
vA[1][2]

tensor([-1.0605, -0.7688,  0.1739, -0.3395,  0.8663,  0.3894,  0.1340, -0.8580,
        -1.6964,  1.0345,  0.6534, -0.6326,  1.1859, -0.2747, -0.0704,  0.5240,
         0.9124,  0.0323,  0.1340,  0.1626, -0.4611,  0.4656,  0.5794,  0.1429,
        -0.9366,  1.1851, -0.6629, -0.7722,  0.4630, -0.8474, -0.8556,  0.2715,
        -0.1746, -0.4981, -0.7126, -0.0603, -0.2227, -0.7841,  0.3040,  0.2132,
         0.3704, -1.0180,  0.5923,  1.4925,  1.0837,  0.3690, -1.0705,  1.0322,
        -0.4234, -0.4439])

In [358]:
matrix[1][2][2]

tensor(-0.1712)

In [407]:
class GaussianKernel(torch.nn.Module):
    def __init__(self, mu: float = 1., sigma: float = 1.):
        super().__init__()
        self.mu = mu
        self.sigma = sigma

    def forward(self, x):
        norm_sq = (x - self.mu)**2
        return np.exp(-norm_sq/(2*self.sigma**2)).sum(1)

In [414]:
gk = GaussianKernel()

s= gk(matrix)

In [416]:
s

tensor([46.6277, 40.5472, 35.1681, 46.8113, 42.9065, 49.8048, 39.1281, 48.8337,
        49.8400, 45.2312, 44.5298, 41.0952, 47.4167, 47.4946, 43.4154, 45.3958,
        46.0130, 45.3535, 35.6942, 48.3856, 40.4608, 50.3658, 50.1863, 49.5398,
        50.7326, 44.7961, 44.4965, 46.6538, 49.8793, 48.1080, 34.9131])

In [91]:
from tqdm import tqdm

seed = 42
min_group_size = 2
fill_top_to = 3

inp_df = s.glue_train_df.copy()
inp_df_select = inp_df[['id_left', 'id_right', 'label']]

In [92]:
t = inp_df_select.merge(inp_df_select, how='left', on='id_left')
tt = t[(t.label_x==1)&(t.label_y==0)]
tt['ones'] = 1
ttt = tt[['id_left', 'id_right_x', 'id_right_y', 'ones']].values




inp_df_select_unr = inp_df_select.copy()
inp_df_select_unr.loc[:,'id_right'] = inp_df_select_unr['id_right'].sample(frac=1, random_state=42).values

unr = inp_df_select_unr.merge(inp_df_select, on=['id_left', 'id_right'], how='left')
unr = unr[unr.label_y.isnull()]

ll = inp_df_select.merge(unr, how='left', on='id_left')
ll['ones'] = 1
lll = ll[['id_left', 'id_right_x', 'id_right_y', 'ones']].values

itog = np.concatenate((ttt,lll))

np.random.seed(seed)
# np.random.shuffle(itog)
itog_ones_ind = np.random.choice(len(itog), size=6000, replace=False)
itog_ones = itog[itog_ones_ind]
itog_ones = itog_ones.tolist()

itog_zeros = []
itog_zeros_ind = np.random.choice(len(itog), size=4000, replace=False)
for i in itog_zeros_ind:
    tl = itog[i]
    q1 = tl[0]
    q2 = tl[2]
    q3 = tl[1]
    itog_zeros.append([q1, q2, q3, 0])

final = itog_ones + itog_zeros
np.random.shuffle(final)

/home/atuthvatullin/environments/albert/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
%%time
s.train(3)